In [0]:
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten
from keras.layers.normalization import BatchNormalization
from keras.callbacks import ModelCheckpoint


# Specify the input shape to the first convolutional layer
input_shape = (120, 160, 3)
nClasses = 2
def createModel():
    model = Sequential()
    # a convolution layer of 32 features of size 3x3 with relu activation and zero padding
    model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=input_shape))
    # a convolution layer of 32 features of size 3x3 with relu activation
    model.add(Conv2D(32, (3, 3), activation='relu'))
    # a batch normalization layer
    model.add(BatchNormalization())
    # maxpooling layer of filter size 2x2
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    # a convolution layer of 64 features of size 3x3 with relu activation and zero padding
    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    # a convolution layer of 64 features of size 3x3 with relu activation
    model.add(Conv2D(64, (3, 3), activation='relu'))
    # a batch normalization layer
    model.add(BatchNormalization())
    # maxpooling layer of filter size 2x2
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    # a convolution layer of 64 features of size 3x3 with relu activation and zero padding
    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    # a convolution layer of 64 features of size 3x3 with relu activation
    model.add(Conv2D(64, (3, 3), activation='relu'))
    # a batch normalization layer
    model.add(BatchNormalization())
    # maxpooling layer of filter size 2x2
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # a dropout layer of 50%
    model.add(Dropout(0.5))
    
    # flatten the output of the previous layer
    model.add(Flatten())
    # add a dense layer that outputs 512 units and apply relu activation
    model.add(Dense(512, activation='relu'))
    # a dropout layer of 50%
    model.add(Dropout(0.5))
    # add a dense layer with a softmax activation to classify the images
    model.add(Dense(nClasses, activation='softmax'))
     
    return model
 

Using TensorFlow backend.


In [0]:
model1 = createModel()
model1.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#### Loading Google Drive

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive



#### 1- Use Keras’ ImageDataGenerator to read the modified dataset and continue training the provided model.
#### 2- Use ImageDataGenerator’s parameters to create validation split (0.2) and to standardize the images.

In [0]:
from keras.preprocessing.image import ImageDataGenerator
# Splitting training data as 0.8 for training and 0.2 for validation.
train_datagen = ImageDataGenerator(
        rescale=1./255,
#         shear_range=0.2,
#         zoom_range=0.2,
#         horizontal_flip=True,
        samplewise_std_normalization=True,
        samplewise_center=True,
        validation_split=0.2)

test_datagen = ImageDataGenerator(
    rescale=1./255,
    samplewise_std_normalization=True,
    samplewise_center=True)




In [0]:
# Reading training set
train_generator = train_datagen.flow_from_directory(
        '/content/gdrive/My Drive/MLProject3/Data/train',
        target_size=(120, 160),
        batch_size=32,
        class_mode='binary',
        subset="training")
#         color_mode="rgb")


Found 3098 images belonging to 2 classes.


In [0]:
# Reading validation set
validation_generator = train_datagen.flow_from_directory(
        '/content/gdrive/My Drive/MLProject3/Data/train',
        target_size=(120, 160),
        batch_size=32,
        class_mode='binary',
        subset="validation"
#         color_mode="rgb"
)


Found 774 images belonging to 2 classes.


In [0]:
# Reading testing set
test_generator = test_datagen.flow_from_directory(
        '/content/gdrive/My Drive/MLProject3/Data/test',
        target_size=(120, 160),
        batch_size=32,
        class_mode='binary'
#         color_mode="rgb"
)


Found 860 images belonging to 2 classes.


#### 4- Load the provided weights file and continue the training. Aim for validation accuracy of 92% at least.

In [0]:
model1.load_weights("/content/gdrive/My Drive/MLProject3/weights.hdf5")

#### 5- Use Keras’ checkpointing support to save the weights that achieve best validation performance. You will need to submit these.

In [0]:
# https://keras.io/callbacks/#modelcheckpoint
weights_save_path = "/content/gdrive/My Drive/MLProject3/best_weights.hdf5"

check_point = ModelCheckpoint(
    weights_save_path, 
    save_best_only=True, 
    save_weights_only=True,
    monitor='acc',
    period=1)

#### 3- Use Keras’ model.fit_generator to train your model. Choose the appropriate steps_per_epoch and validation_steps value. These values are related to the batch size. 

#### Justify your choice.


* The steps_per_epoch should typically be equal to the number of samples of your dataset divided by the batch size. So the number of data sample of the data set is train_generator.n and the batch size is train_generator.batch_size (32). Then I divided them by each other.

* e.g. If a dataset has 1000 images and a batch size of 10, then the steps_per_epoch will be = 1000/10 = 100. Meaning that there will be 100 batches done per epoch (whole dataset).

* Same goes for the validation_steps.

* The validation_steps should typically be equal to the number of samples of your validation dataset divided by the batch size. So the number of data sample of the data set is validation_generator.n and the batch size is validation_generator.batch_size (32). Then I divided them by each other.

* I have added a checkpoint to the list of callbacks to save the best weights.

In [0]:
STEP_SIZE_TRAIN = train_generator.n // train_generator.batch_size
STEP_SIZE_VALID = validation_generator.n // validation_generator.batch_size

print(STEP_SIZE_TRAIN)
print(len(train_generator))
print(len(validation_generator))
print(len(test_generator))

96
97
25
27


In [0]:
history = model1.fit_generator(
            train_generator,
            validation_data=validation_generator,
            steps_per_epoch=STEP_SIZE_TRAIN, 
            verbose=1,
            epochs=30,
            validation_steps=STEP_SIZE_VALID, 
            callbacks=[check_point])

Epoch 1/30
96/96 [==============================] - 17s 179ms/step - loss: 0.5770 - acc: 0.8629 - val_loss: 0.6410 - val_acc: 0.8706
Epoch 2/30
96/96 [==============================] - 16s 162ms/step - loss: 0.3736 - acc: 0.8869 - val_loss: 0.7096 - val_acc: 0.8518
Epoch 3/30
96/96 [==============================] - 15s 161ms/step - loss: 0.3293 - acc: 0.9071 - val_loss: 1.0979 - val_acc: 0.8221
Epoch 4/30
96/96 [==============================] - 15s 161ms/step - loss: 0.4139 - acc: 0.9046 - val_loss: 1.4401 - val_acc: 0.8315
Epoch 5/30
96/96 [==============================] - 16s 168ms/step - loss: 0.3250 - acc: 0.9133 - val_loss: 0.6037 - val_acc: 0.8908
Epoch 6/30
96/96 [==============================] - 16s 166ms/step - loss: 0.2938 - acc: 0.9304 - val_loss: 5.4769 - val_acc: 0.6348
Epoch 7/30
96/96 [==============================] - 16s 165ms/step - loss: 0.2771 - acc: 0.9368 - val_loss: 0.9436 - val_acc: 0.8342
Epoch 8/30
96/96 [==============================] - 16s 165ms/step - 

In [0]:
print(history.history.keys())

dict_keys(['val_loss', 'val_acc', 'loss', 'acc'])


#### 6- Evaluate your model on the test set provided.

In [0]:
STEP_SIZE_TEST = test_generator.n//test_generator.batch_size

scores = model1.evaluate_generator(test_generator, verbose=1, steps=STEP_SIZE_TEST)

# print(model1.metrics_names)
# print(scores)
print("\n")
print("Validation Accurary: %.2f%%" % max(history.history['val_acc']))
print()
print("Testing Accuracy: %.2f%%" % scores[1])

26/26 [==============================] - 3s 119ms/step


Validation Accurary: 0.93%

Testing Accuracy: 0.96%
